In [ ]:
!pip install neo4j
!pip3 install tomlkit
!pip3 install -U neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.19.0-py3-none-any.whl size=280741 sha256=53f725d535e4a621789e071fbab26387f6e12e4fa0b6b22f69d2710a1353787e
  Stored in directory: /root/.cache/pip/wheels/30/db/9b/2cfde1fa33145219c0322f299b604daf5aba2ed443a7ed5f07
Successfully built neo4j


In [ ]:
from neo4j import GraphDatabase

database_name = "neo4j"
username = "neo4j"
password = "apan5400"
uri = "bolt://localhost:7687/apan5400" + database_name

driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

print("Successfully connected to Neo4j!")


Successfully connected to Neo4j!


In [ ]:
import json
import pandas as pd
import os
import urllib

file_path = "C:/Users/pulki/Downloads/combined_data.jsonl"
data = []

df = pd.DataFrame(data)

# Define the Cypher query
cypher_query = '''
UNWIND $products AS product
MERGE (p:Product {asin: product.parent_asin})
  ON CREATE SET p.title = product.title, p.price = product.price
MERGE (u:User {id: product.user_id})
MERGE (u)-[:PURCHASED {rating: product.rating}]->(p)
MERGE (s:Store {name: product.store})
MERGE (p)-[:SOLD_BY]->(s)
WITH product, p
UNWIND product.categories AS category
MERGE (c:Category {name: category})
MERGE (p)-[:BELONGS_TO]->(c)
'''

# Convert DataFrame to a list of dictionaries
products = df.to_dict('records')

# Create a Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password), encrypted=False)

# Execute the Cypher query
execute_query(driver, cypher_query, parameters={'products': products})

print("Graph creation in Neo4j is complete.")

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address)

In [ ]:
#The plan:

#Product Nodes: Each product can be a node, with properties like title, average_rating, price, etc.
#User Nodes: Each user can also be a node, with a user_id property.
#Category Nodes: Categories can be nodes as well, with the category name as a property.
#Store Nodes: Stores are also nodes, with the store name as a property.
#PURCHASED Relationships: Connect User nodes to Product nodes to represent a purchase. The rating could be a property of this relationship.
#BELONGS_TO Relationship: Connect Product nodes to Category nodes to represent the categories each product belongs to.
#SOLD_BY Relationship: Connect Product nodes to Store nodes to represent the store selling the product.